# MNIST Database

The MNIST database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems. Widely used for training and testing in the field of machine learning.

- images of upper & lower case letters plus digits
- 28x28 pixel
- grayscale
- ~70,000 labelled images.

## Loading this via `torchvision`

A pytorch extension that consists of popular datasets, yet critically baking into them the separation of training and testing datasets to enforce a bit more consistency across research, as everyone uses the same split - allows for comparable benchmarking of models.

`torchvision` allows you to "transform" your data as you load it in, which is typically required as you interface your dataset with your NN. We will explicitly avoid the use of this utility to describe nessecary transformations explicitly later. I'm still learning after all.

In [5]:
from torchvision import datasets

train_dataset = datasets.MNIST('./data', train=True, download=True)
test_dataset = datasets.MNIST('./data', train=False)

In [8]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train

### Mini-batch SGD will be our training approach
The `DataLoader` class in `pytorch` conveniently handles the the stochastic, mini-batch sourcing of our training. Note;

The choice of batch size is a trade-off:

- Larger batches provide a more accurate estimate of the gradient but require more memory and can lead to a less exploratory update path, potentially getting stuck in shallow local minima.
- Smaller batches offer more update opportunities and better generalization at the cost of increased noise in the updates and potentially longer training times.

In [7]:
import torch

batch_size = 2^6

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Re-introducing you to typical NN's

### `pytorch`
The SOTA ML framework today that is built upon on the early lua implementation of [the torch library](https://en.wikipedia.org/wiki/Torch_(machine_learning)). Came in during 2017 and has unequivocally been the top tool for ML research.

`pytorch` comes with a comprehensive `nn` package that handles common activation functions and architectures for any NN's.

#### Input Layer
Considering the 28x28 pixel, where each pixel is grayscale restricted, intensity can be presented via a value between 0 and 255, where 0 represents black and 255 represents white.

ModuleNotFoundError: No module named 'torch'